In [1]:
import os
import pandas as pd
os.chdir('/Users/rishi/Desktop/DAEN 690/glassdoor')

In [2]:
#to process the text
import nltk
from nltk.tokenize import sent_tokenize
#pos tagging
from nltk import pos_tag
import string
from nltk.corpus import stopwords
#to create frequency distribution
from nltk.probability import FreqDist
#to create lemmas of the words
from nltk.stem import WordNetLemmatizer
#to perform operations using regular expressions
import re

In [3]:
#to generate ngrams of text
from nltk.util import ngrams

In [4]:
whole_df=pd.read_csv('Client Descriptions.csv')

In [5]:
whole_df['Description'] = whole_df['Description'].astype(str)

In [6]:
#whole_df['Description'] = whole_df['Description'].str.lstrip('0')

In [7]:
whole_df

,Labor Category,Description
0,IT Project Manager I,Manages projects and development teams executi...
1,IT Project Manager III,Manages projects and development teams executi...
2,Senior Computer Security Systems Specialist,Analyzes and defines security requirements for...
3,Senior Security Analyst,Analyzes security measures for more than one I...
4,Cloud Engineer,Experience with cloud services - including ope...
5,Senior Data Scientist,Data Scientist will have necessary statistical...
6,User Experience (UX) Developer,Responsible for creating front-end design solu...
7,Software Developer I,"Develop, modify, or update applications used b..."
8,Software Developer III,"Develop, modify, or update applications used b..."
9,Test Automation Engineer,The Test Automation Engineer is responsible fo...


In [8]:
#Text Preprocessing
#Sentence tokenization
def tokenize_sentences(textContent):
    #No of sentences
    sentences = nltk.sent_tokenize(textContent.lower())
    return sentences

#Word Tokenization
def tokenize_words(textContent):
    tokenized_words = nltk.word_tokenize(textContent)
    return tokenized_words

#remove punctuations
def remove_punctuation(textContent):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    punc_removed_words = tokenizer.tokenize(textContent.lower())
    return punc_removed_words

#remove stop words
#In this scenario, pass words obtained after removing punctuation
def remove_stopwords(word_list):
    stopwords_removed = []  
    stop_words = set(stopwords.words('english'))
    for w in word_list:  
        if w not in stop_words:  
            stopwords_removed.append(w) 
    return stopwords_removed

#Remove digits
#In this scenario, pass word list obtained after removing stop words
#def remove_digits_from_words(word_list):
#    digits_removed = []
#    for w in word_list:
#        word = re.sub(r'[0-9]+', '', w)
#        digits_removed.append(word)
#   return digits_removed

#lemmatization
#in this scenario, pass words list obtained after removing stopwords
def create_lemmas(word_list):
    lemmatizer = WordNetLemmatizer() 
    lemmatized_words = []
   
    for word, tag in pos_tag(word_list):
        wntag = tag[0].lower()
        #POS tags 
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        lemma = lemmatizer.lemmatize(word, wntag) if wntag else word
        lemmatized_words.append(lemma)
    return lemmatized_words

In [9]:
#Perform Text processing
def get_preprocessed_content(row, text_from, text_preprocess_category):
    
    textContent = row[text_from]
    if text_preprocess_category == 'sentences':
        sentence_tokens = tokenize_sentences(textContent)
        return sentence_tokens
    elif text_preprocess_category == 'word_tokens':
        w_tokens = tokenize_words(textContent)
        return w_tokens
    elif text_preprocess_category == 'words_no_punctuation':
        no_punc_words = remove_punctuation(textContent)
        return no_punc_words
    else:
        return None

In [10]:
#get tokens/ stopwords/ lemmas
def get_preprocessed_tokens(row, tokens_from, text_preprocess_category):
    
    token_list = row[tokens_from]
    
    if text_preprocess_category == 'words_no_sw':
        no_stop_words = remove_stopwords(token_list)
        return no_stop_words
    elif text_preprocess_category == 'words_no_digits':
        no_digits = remove_digits_from_words(token_list)
        return no_digits
    elif text_preprocess_category == 'lemmas':
        lemmas = create_lemmas(token_list)
        return lemmas
    else:
        return None

In [11]:
preprocessing_actions_one = ['sentences','word_tokens','words_no_punctuation']
preprocessing_actions_two = ['words_no_sw','lemmas']

In [12]:
# for total data
whole_title_df = pd.DataFrame()

for pa in preprocessing_actions_one:
    whole_title_df[pa] = whole_df.apply(lambda row: get_preprocessed_content(row, 'Description',pa), axis=1)
    
for pa in preprocessing_actions_two:
    if pa == 'words_no_sw':
        tokens_from = 'words_no_punctuation'
    elif pa == 'lemmas':
        tokens_from = 'words_no_sw'
    whole_title_df[pa] = whole_title_df.apply(lambda row: get_preprocessed_tokens(row,tokens_from ,pa), axis=1)

In [13]:
whole_title_df

,sentences,word_tokens,words_no_punctuation,words_no_sw,lemmas
0,[manages projects and development teams execut...,"[Manages, projects, and, development, teams, e...","[manages, projects, and, development, teams, e...","[manages, projects, development, teams, execut...","[manages, project, development, team, execute,..."
1,[manages projects and development teams execut...,"[Manages, projects, and, development, teams, e...","[manages, projects, and, development, teams, e...","[manages, projects, development, teams, execut...","[manages, project, development, team, execute,..."
2,[analyzes and defines security requirements fo...,"[Analyzes, and, defines, security, requirement...","[analyzes, and, defines, security, requirement...","[analyzes, defines, security, requirements, mu...","[analyzes, defines, security, requirement, mul..."
3,[analyzes security measures for more than one ...,"[Analyzes, security, measures, for, more, than...","[analyzes, security, measures, for, more, than...","[analyzes, security, measures, one, functional...","[analyzes, security, measure, one, functional,..."
4,[experience with cloud services - including op...,"[Experience, with, cloud, services, -, includi...","[experience, with, cloud, services, including,...","[experience, cloud, services, including, open,...","[experience, cloud, service, include, open, so..."
5,[data scientist will have necessary statistica...,"[Data, Scientist, will, have, necessary, stati...","[data, scientist, will, have, necessary, stati...","[data, scientist, necessary, statistical, mode...","[data, scientist, necessary, statistical, mode..."
6,[responsible for creating front-end design sol...,"[Responsible, for, creating, front-end, design...","[responsible, for, creating, front, end, desig...","[responsible, creating, front, end, design, so...","[responsible, create, front, end, design, solu..."
7,"[develop, modify, or update applications used ...","[Develop, ,, modify, ,, or, update, applicatio...","[develop, modify, or, update, applications, us...","[develop, modify, update, applications, used, ...","[develop, modify, update, application, use, bu..."
8,"[develop, modify, or update applications used ...","[Develop, ,, modify, ,, or, update, applicatio...","[develop, modify, or, update, applications, us...","[develop, modify, update, applications, used, ...","[develop, modify, update, application, use, bu..."
9,[the test automation engineer is responsible f...,"[The, Test, Automation, Engineer, is, responsi...","[the, test, automation, engineer, is, responsi...","[test, automation, engineer, responsible, anal...","[test, automation, engineer, responsible, anal..."


In [14]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))
        
        
# Driver code    
whole_title_df['full']= whole_title_df.apply(lambda row: listToString(row['lemmas']), axis=1)


In [15]:
whole_title_df

,sentences,word_tokens,words_no_punctuation,words_no_sw,lemmas,full
0,[manages projects and development teams execut...,"[Manages, projects, and, development, teams, e...","[manages, projects, and, development, teams, e...","[manages, projects, development, teams, execut...","[manages, project, development, team, execute,...",manages project development team execute range...
1,[manages projects and development teams execut...,"[Manages, projects, and, development, teams, e...","[manages, projects, and, development, teams, e...","[manages, projects, development, teams, execut...","[manages, project, development, team, execute,...",manages project development team execute range...
2,[analyzes and defines security requirements fo...,"[Analyzes, and, defines, security, requirement...","[analyzes, and, defines, security, requirement...","[analyzes, defines, security, requirements, mu...","[analyzes, defines, security, requirement, mul...",analyzes defines security requirement multilev...
3,[analyzes security measures for more than one ...,"[Analyzes, security, measures, for, more, than...","[analyzes, security, measures, for, more, than...","[analyzes, security, measures, one, functional...","[analyzes, security, measure, one, functional,...",analyzes security measure one functional area ...
4,[experience with cloud services - including op...,"[Experience, with, cloud, services, -, includi...","[experience, with, cloud, services, including,...","[experience, cloud, services, including, open,...","[experience, cloud, service, include, open, so...",experience cloud service include open source t...
5,[data scientist will have necessary statistica...,"[Data, Scientist, will, have, necessary, stati...","[data, scientist, will, have, necessary, stati...","[data, scientist, necessary, statistical, mode...","[data, scientist, necessary, statistical, mode...",data scientist necessary statistical model mat...
6,[responsible for creating front-end design sol...,"[Responsible, for, creating, front-end, design...","[responsible, for, creating, front, end, desig...","[responsible, creating, front, end, design, so...","[responsible, create, front, end, design, solu...",responsible create front end design solution w...
7,"[develop, modify, or update applications used ...","[Develop, ,, modify, ,, or, update, applicatio...","[develop, modify, or, update, applications, us...","[develop, modify, update, applications, used, ...","[develop, modify, update, application, use, bu...",develop modify update application use business...
8,"[develop, modify, or update applications used ...","[Develop, ,, modify, ,, or, update, applicatio...","[develop, modify, or, update, applications, us...","[develop, modify, update, applications, used, ...","[develop, modify, update, application, use, bu...",develop modify update application use business...
9,[the test automation engineer is responsible f...,"[The, Test, Automation, Engineer, is, responsi...","[the, test, automation, engineer, is, responsi...","[test, automation, engineer, responsible, anal...","[test, automation, engineer, responsible, anal...",test automation engineer responsible analysis ...


In [16]:
#Ngram
def generate_ngrams(text,n):
    n_grams = ngrams(nltk.word_tokenize(text.lower()), n)
    return [' '.join(grams) for grams in n_grams]

whole_title_df['tri-grams'] = whole_title_df.apply(lambda row: generate_ngrams(row['full'],3), axis=1)
whole_title_df['bi-grams'] = whole_title_df.apply(lambda row: generate_ngrams(row['full'],2), axis=1)

In [17]:
whole_title_df

,sentences,word_tokens,words_no_punctuation,words_no_sw,lemmas,full,tri-grams,bi-grams
0,[manages projects and development teams execut...,"[Manages, projects, and, development, teams, e...","[manages, projects, and, development, teams, e...","[manages, projects, development, teams, execut...","[manages, project, development, team, execute,...",manages project development team execute range...,"[manages project development, project developm...","[manages project, project development, develop..."
1,[manages projects and development teams execut...,"[Manages, projects, and, development, teams, e...","[manages, projects, and, development, teams, e...","[manages, projects, development, teams, execut...","[manages, project, development, team, execute,...",manages project development team execute range...,"[manages project development, project developm...","[manages project, project development, develop..."
2,[analyzes and defines security requirements fo...,"[Analyzes, and, defines, security, requirement...","[analyzes, and, defines, security, requirement...","[analyzes, defines, security, requirements, mu...","[analyzes, defines, security, requirement, mul...",analyzes defines security requirement multilev...,"[analyzes defines security, defines security r...","[analyzes defines, defines security, security ..."
3,[analyzes security measures for more than one ...,"[Analyzes, security, measures, for, more, than...","[analyzes, security, measures, for, more, than...","[analyzes, security, measures, one, functional...","[analyzes, security, measure, one, functional,...",analyzes security measure one functional area ...,"[analyzes security measure, security measure o...","[analyzes security, security measure, measure ..."
4,[experience with cloud services - including op...,"[Experience, with, cloud, services, -, includi...","[experience, with, cloud, services, including,...","[experience, cloud, services, including, open,...","[experience, cloud, service, include, open, so...",experience cloud service include open source t...,"[experience cloud service, cloud service inclu...","[experience cloud, cloud service, service incl..."
5,[data scientist will have necessary statistica...,"[Data, Scientist, will, have, necessary, stati...","[data, scientist, will, have, necessary, stati...","[data, scientist, necessary, statistical, mode...","[data, scientist, necessary, statistical, mode...",data scientist necessary statistical model mat...,"[data scientist necessary, scientist necessary...","[data scientist, scientist necessary, necessar..."
6,[responsible for creating front-end design sol...,"[Responsible, for, creating, front-end, design...","[responsible, for, creating, front, end, desig...","[responsible, creating, front, end, design, so...","[responsible, create, front, end, design, solu...",responsible create front end design solution w...,"[responsible create front, create front end, f...","[responsible create, create front, front end, ..."
7,"[develop, modify, or update applications used ...","[Develop, ,, modify, ,, or, update, applicatio...","[develop, modify, or, update, applications, us...","[develop, modify, update, applications, used, ...","[develop, modify, update, application, use, bu...",develop modify update application use business...,"[develop modify update, modify update applicat...","[develop modify, modify update, update applica..."
8,"[develop, modify, or update applications used ...","[Develop, ,, modify, ,, or, update, applicatio...","[develop, modify, or, update, applications, us...","[develop, modify, update, applications, used, ...","[develop, modify, update, application, use, bu...",develop modify update application use business...,"[develop modify update, modify update applicat...","[develop modify, modify update, update applica..."
9,[the test automation engineer is responsible f...,"[The, Test, Automation, Engineer, is, responsi...","[the, test, automation, engineer, is, responsi...","[test, automation, enginee

In [18]:
import itertools


In [19]:
ngram = whole_title_df['tri-grams']
ngram = list(itertools.chain.from_iterable(ngram))

In [20]:
from nltk import FreqDist


In [21]:
fdist = nltk.FreqDist(ngram)

In [22]:
def find_weighted_frequency(fdist):
    FreqDist = {}
    
    for word in fdist.keys():
        FreqDist[word] = fdist.get(word)
        
    max_freq = fdist.most_common(1)[0][1]
    total_words = max(fdist.values())   
    
    for word in FreqDist.keys():
        FreqDist[word] = (FreqDist[word]/max_freq)
    return FreqDist


ngram_freqs = find_weighted_frequency(fdist)
list (ngram_freqs.items())[0:20]

[('manages project development', 1.0),
 ('project development team', 1.0),
 ('development team execute', 1.0),
 ('team execute range', 1.0),
 ('execute range methodology', 1.0),
 ('range methodology include', 1.0),
 ('methodology include waterfall', 1.0),
 ('include waterfall agile', 1.0),
 ('waterfall agile lean', 1.0),
 ('agile lean ensures', 1.0),
 ('lean ensures project', 1.0),
 ('ensures project meet', 1.0),
 ('project meet scope', 1.0),
 ('meet scope schedule', 1.0),
 ('scope schedule budget', 1.0),
 ('schedule budget serve', 1.0),
 ('budget serve scrum', 1.0),
 ('serve scrum master', 1.0),
 ('scrum master agile', 1.0),
 ('master agile project', 1.0)]